<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/TopicModeling_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
doc_ls = ['바나나 사과 포도 포도 짜장면', '사과 포도', '포도 바나나', '짜장면 짬뽕 탕수육', '볶음밥 탕수육',
          '짜장면 짬뽕', '라면 스시', '스시 짜장면', '가츠동 스시 소바', '된장찌개 김치찌개 김치', 
          '김치 된장 짜장면', '비빔밥 김치']
    

### TDM 생성 (문서-단어 행렬)

In [19]:
import numpy as np
import pandas as pd

In [20]:
total_token = (" ".join(doc_ls)).split(" ")
total_unique_token = list(np.unique(total_token))

In [21]:
tokenized_doc_ls = [doc.split(" ") for doc in doc_ls]
tokenized_doc_ls

[['바나나', '사과', '포도', '포도', '짜장면'],
 ['사과', '포도'],
 ['포도', '바나나'],
 ['짜장면', '짬뽕', '탕수육'],
 ['볶음밥', '탕수육'],
 ['짜장면', '짬뽕'],
 ['라면', '스시'],
 ['스시', '짜장면'],
 ['가츠동', '스시', '소바'],
 ['된장찌개', '김치찌개', '김치'],
 ['김치', '된장', '짜장면'],
 ['비빔밥', '김치']]

In [22]:
token_count_in_doc = []
for doc in tokenized_doc_ls:
    tmp = []
    for token in total_unique_token:
        tmp.append(doc.count(token))
    token_count_in_doc.append(tmp)
tdm = pd.DataFrame(token_count_in_doc, index=['문서{}'.format(i) for i in range(len(doc_ls))], columns=total_unique_token)
tdm

,가츠동,김치,김치찌개,된장,된장찌개,라면,바나나,볶음밥,비빔밥,사과,소바,스시,짜장면,짬뽕,탕수육,포도
문서0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,2
문서1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
문서2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
문서3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
문서4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
문서5,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
문서6,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
문서7,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
문서8,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
문서9,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0


## 특이값 분해(SVD)

In [23]:
A = np.array(token_count_in_doc)

In [19]:
U, s, VT = np.linalg.svd(A)


In [25]:
U.shape

(12, 12)

In [26]:
s.shape

(12,)

In [27]:
VT.shape

(16, 16)

## 토픽모델링: n개의 토픽

In [24]:
def _model_topic(A, n_topics, n_words): #행렬A에서 n_topics개의 토픽 추출, 토픽당 n_words개의 단어를 추출
    U, s, VT = np.linalg.svd(A)
    for topic in range(n_topics) :
        sort = np.argsort(VT.T[:,topic])[::-1]
        top = sort[0:n_words]

        words_group = []
        for i in top :
            words_group.append((total_unique_token[i],  '%.3f' %VT.T[:,topic][i]))
        print("Topic{}: {}".format(topic+1, words_group))



In [29]:
def _model_topic(A, n_topics, n_words): #행렬A에서 n_topics개의 토픽 추출, 토픽당 n_words개의 단어를 추출
    U, s, VT = np.linalg.svd(A)
    for topic in range(n_topics) :
        sort = np.argsort(VT.T[:,topic])[::-1]
        top = sort[0:n_words]

        words_group = []
        for i in top :
            words_group.append((total_unique_token[i],  '%.3f' %VT.T[:,topic][i]))
        print("Topic{}: {}".format(topic+1, words_group))


In [30]:
_model_topic(A, 4, 3)

Topic1: [('포도', '0.697'), ('짜장면', '0.486'), ('사과', '0.348')]
Topic2: [('포도', '0.387'), ('사과', '0.194'), ('바나나', '0.194')]
Topic3: [('스시', '0.523'), ('소바', '0.226'), ('가츠동', '0.226')]
Topic4: [('스시', '0.552'), ('김치', '0.371'), ('소바', '0.277')]


## Class

In [31]:
import numpy as np
import pandas as pd

class TopicModeling:
    def __init__(self, docs):
        self.docs = docs
        self.total_unique_token = []

    def _make_matrix(self):
        total_token = (" ".join(self.docs)).split(" ")
        self.total_unique_token = list(np.unique(total_token))
        tokenized_doc_ls = [doc.split(" ") for doc in doc_ls]
        
        token_count_in_doc = []
        for doc in tokenized_doc_ls:
            tmp = []
            for token in self.total_unique_token:
                tmp.append(doc.count(token))
            token_count_in_doc.append(tmp)        
        return np.array(token_count_in_doc)

    def model_topic(self, n_topics, n_words): #행렬A에서 n_topics개의 토픽 추출, 토픽당 n_words개의 단어를 추출
        A = self._make_matrix()
        U, s, VT = np.linalg.svd(A)
        for topic in range(n_topics) :
            sort = np.argsort(VT.T[:,topic])[::-1]
            top = sort[0:n_words]

            words_group = []
            for i in top :
                words_group.append((self.total_unique_token[i], '%.3f' %VT.T[:,topic][i]))
            print("Topic{}: {}".format(topic+1, words_group))
    

In [32]:
topicmodel = TopicModeling(docs=doc_ls)

In [33]:
topicmodel.model_topic(4,4)

Topic1: [('포도', '0.697'), ('짜장면', '0.486'), ('사과', '0.348'), ('바나나', '0.348')]
Topic2: [('포도', '0.387'), ('사과', '0.194'), ('바나나', '0.194'), ('볶음밥', '-0.050')]
Topic3: [('스시', '0.523'), ('소바', '0.226'), ('가츠동', '0.226'), ('라면', '0.158')]
Topic4: [('스시', '0.552'), ('김치', '0.371'), ('소바', '0.277'), ('가츠동', '0.277')]
